<a href="https://colab.research.google.com/github/VictorMadu/Coursera-Deep-Learning-Specialization-Course/blob/master/How-to-win-a-data-science-competition/Week4/compute_KNN_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Version 1.1.1

# The task

In this assignment you will need to implement features, based on nearest neighbours. 

KNN classifier (regressor) is a very powerful model, when the features are homogeneous and it is a very common practice to use KNN as first level model. In this homework we will extend KNN model and compute more features, based on nearest neighbors and their distances. 

You will need to implement a number of features, that were one of the key features, that leaded the instructors to prizes in [Otto](https://www.kaggle.com/c/otto-group-product-classification-challenge) and [Springleaf](https://www.kaggle.com/c/springleaf-marketing-response) competitions. Of course, the list of features you will need to implement can be extended, in fact in competitions the list was at least 3 times larger. So when solving a real competition do not hesitate to make up your own features.   

You can optionally implement multicore feature computation. Nearest neighbours are hard to compute so it is preferable to have a parallel version of the algorithm. In fact, it is really a cool skill to know how to use `multiprocessing`, `joblib` and etc. In this homework you will have a chance to see the benefits of parallel algorithm. 

# Check your versions


Some functions we use here are not present in old versions of the libraries, so make sure you have up-to-date software. 

In [ ]:
!pip install numpy==1.13.1
!pip install scipy==0.19.1
!pip install scikit-learn==0.19.0
!pip install pandas==0.20.3

     |████████████████████████████████| 17.0MB 1.3MB/s 
ERROR: yellowbrick 0.9.1 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
ERROR: xarray 0.15.1 has requirement numpy>=1.15, but you'll have numpy 1.13.1 which is incompatible.
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.13.1 which is incompatible.
ERROR: umap-learn 0.4.6 has requirement scipy>=1.3.1, but you'll have scipy 0.19.1 which is incompatible.
ERROR: tifffile 2020.7.24 has requirement numpy>=1.15.1, but you'll have numpy 1.13.1 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.13.1 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 0.19.1 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement numpy>=1.13.3, but you'll have numpy 1.13.1 which is incompatible.
ERROR: spacy 2.2.4 has requirement numpy>=1.15.0, but you'll have numpy 1.

     |████████████████████████████████| 12.4MB 327kB/s 
ERROR: yellowbrick 0.9.1 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.19.0 which is incompatible.
ERROR: yellowbrick 0.9.1 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.13.1 which is incompatible.
ERROR: umap-learn 0.4.6 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.19.0 which is incompatible.
ERROR: umap-learn 0.4.6 has requirement scipy>=1.3.1, but you'll have scipy 0.19.1 which is incompatible.
ERROR: librosa 0.6.3 has requirement scikit-learn!=0.19.0,>=0.14.0, but you'll have scikit-learn 0.19.0 which is incompatible.
ERROR: librosa 0.6.3 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
ERROR: imbalanced-learn 0.4.3 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.19.0 which is incompatible.
ERROR: fancyimpute 0.4.3 has req

In [ ]:
import numpy as np
import pandas as pd 
import sklearn
import scipy.sparse 

for p in [np, pd, sklearn, scipy]:
    print (p.__name__, p.__version__)

numpy 1.13.1
pandas 0.20.3
sklearn 0.19.0
scipy 0.19.1


The versions should be not less than:

    numpy 1.13.1
    pandas 0.20.3
    sklearn 0.19.0
    scipy 0.19.1
   
**IMPORTANT!** The results with `scipy=1.0.0` will be different! Make sure you use _exactly_ version `0.19.1`.

# Load data

Learn features and labels. These features are actually OOF predictions of linear models.

In [ ]:
! wget https://github.com/hse-aml/competitive-data-science/blob/master/readonly/KNN_features_data/X.npz?raw=true -O X.npz
! wget https://github.com/hse-aml/competitive-data-science/blob/master/readonly/KNN_features_data/X_test.npz?raw=true -O X_test.npz
! wget https://github.com/hse-aml/competitive-data-science/blob/master/readonly/KNN_features_data/Y.npy?raw=true -O Y.npy
! wget https://github.com/hse-aml/competitive-data-science/blob/master/readonly/KNN_features_data/Y_test.npy?raw=true -O Y_test.npy
! wget https://github.com/hse-aml/competitive-data-science/blob/master/readonly/KNN_features_data/knn_feats_test_first50.npy?raw=true -O knn_feats_test_first50.npy

--2020-08-18 16:38:31--  https://github.com/hse-aml/competitive-data-science/blob/master/readonly/KNN_features_data/X.npz?raw=true
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/hse-aml/competitive-data-science/raw/master/readonly/KNN_features_data/X.npz [following]
--2020-08-18 16:38:31--  https://github.com/hse-aml/competitive-data-science/raw/master/readonly/KNN_features_data/X.npz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hse-aml/competitive-data-science/master/readonly/KNN_features_data/X.npz [following]
--2020-08-18 16:38:31--  https://raw.githubusercontent.com/hse-aml/competitive-data-science/master/readonly/KNN_features_data/X.npz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.

In [ ]:
train_path = 'X.npz'
train_labels = 'Y.npy'

test_path = 'X_test.npz'
test_labels = 'Y_test.npy'

# Train data
X = scipy.sparse.load_npz(train_path)
Y = np.load(train_labels)

# Test data
X_test = scipy.sparse.load_npz(test_path)
Y_test = np.load(test_labels)

# Out-of-fold features we loaded above were generated with n_splits=4 and skf seed 123
# So it is better to use seed 123 for generating KNN features as well 
skf_seed = 123
n_splits = 4

Below you need to implement features, based on nearest neighbors.

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import NearestNeighbors
from multiprocessing import Pool

import numpy as np


class NearestNeighborsFeats(BaseEstimator, ClassifierMixin):
    '''
        This class should implement KNN features extraction 
    '''
    def __init__(self, n_jobs, k_list, metric, n_classes=None, n_neighbors=None, eps=1e-6):
        self.n_jobs = n_jobs
        self.k_list = k_list
        self.metric = metric
        
        if n_neighbors is None:
            self.n_neighbors = max(k_list) 
        else:
            self.n_neighbors = n_neighbors
            
        self.eps = eps        
        self.n_classes_ = n_classes
    
    def fit(self, X, y):
        '''
            Set's up the train set and self.NN object
        '''
        # Create a NearestNeighbors (NN) object. We will use it in `predict` function 
        self.NN = NearestNeighbors(n_neighbors=max(self.k_list), 
                                      metric=self.metric, 
                                      n_jobs=1, 
                                      algorithm='brute' if self.metric=='cosine' else 'auto')
        self.NN.fit(X)
        
        # Store labels 
        self.y_train = y
        
        # Save how many classes we have
        self.n_classes = np.unique(y).shape[0] if self.n_classes_ is None else self.n_classes_
        
        
    def predict(self, X):       
        '''
            Produces KNN features for every object of a dataset X
        '''
        if self.n_jobs == 1:
            test_feats = []
            for i in range(X.shape[0]):
                test_feats.append(self.get_features_for_one(X[i:i+1]))
        else:
            '''
                 *Make it parallel*
                     Number of threads should be controlled by `self.n_jobs`  
                     
                     
                     You can use whatever you want to do it
                     For Python 3 the simplest option would be to use 
                     `multiprocessing.Pool` (but don't use `multiprocessing.dummy.Pool` here)
                     You may try use `joblib` but you will most likely encounter an error, 
                     that you will need to google up (and eventually it will work slowly)
                     
                     For Python 2 I also suggest using `multiprocessing.Pool` 
                     You will need to use a hint from this blog 
                     http://qingkaikong.blogspot.ru/2016/12/python-parallel-method-in-class.html
                     I could not get `joblib` working at all for this code 
                     (but in general `joblib` is very convenient)
                     
            '''
            
            # YOUR CODE GOES HERE
            # test_feats =  # YOUR CODE GOES HERE
            test_feats = Pool(self.n_jobs).map(self.get_features_for_one,
                                              (X[i:i+1] for i in range(X.shape[0])))
            # YOUR CODE GOES HERE
            
#             assert False, 'You need to implement it for n_jobs > 1'
            
            
            
        return np.vstack(test_feats)
        
        
    def get_features_for_one(self, x):
        '''
            Computes KNN features for a single object `x`
        '''

        NN_output = self.NN.kneighbors(x)
        
        # Vector of size `n_neighbors`
        # Stores indices of the neighbors
        neighs = NN_output[1][0]
        
        # Vector of size `n_neighbors`
        # Stores distances to corresponding neighbors
        neighs_dist = NN_output[0][0] 

        # Vector of size `n_neighbors`
        # Stores labels of corresponding neighbors
        neighs_y = self.y_train[neighs] 
        
        ## ========================================== ##
        ##              YOUR CODE BELOW
        ## ========================================== ##
        
        # We will accumulate the computed features here
        # Eventually it will be a list of lists or np.arrays
        # and we will use np.hstack to concatenate those
        return_list = [] 
        
        
        ''' 
            1. Fraction of objects of every class.
               It is basically a KNNСlassifiers predictions.

               Take a look at `np.bincount` function, it can be very helpful
               Note that the values should sum up to one
        '''
        for k in self.k_list:
            # YOUR CODE GOES HERE
            feats = np.bincount(neighs_y[:k], minlength=self.n_classes) / k
            
            assert len(feats) == self.n_classes
            return_list += [feats]
        
        
        '''
            2. Same label streak: the largest number N, 
               such that N nearest neighbors have the same label.
               
               What can help you: `np.where`
        '''
        
        feats = 1 + \
                np.where(np.append(neighs_y[:-1] != neighs_y[1:], True))[0].min(keepdims=True)# YOUR CODE GOES HERE
        
        assert len(feats) == 1
        return_list += [feats]
        
        '''
            3. Minimum distance to objects of each class
               Find the first instance of a class and take its distance as features.
               
               If there are no neighboring objects of some classes, 
               Then set distance to that class to be 999.

               `np.where` might be helpful
        '''
        feats = []
        for c in range(self.n_classes):
            feats.append(np.append(neighs_dist[neighs_y == c], 999).min())# YOUR CODE GOES HERE
        
        assert len(feats) == self.n_classes
        return_list += [feats]
        
        '''
            4. Minimum *normalized* distance to objects of each class
               As 3. but we normalize (divide) the distances
               by the distance to the closest neighbor.
               
               If there are no neighboring objects of some classes, 
               Then set distance to that class to be 999.
               
               Do not forget to add self.eps to denominator.
        '''
        feats = []
        for c in range(self.n_classes):
            # YOUR CODE GOES HERE
            feat = neighs_dist[neighs_y == c] / (neighs_dist[0] + self.eps)
            feats.append(feat.min() if feat.size else 999)
        
        assert len(feats) == self.n_classes
        return_list += [feats]
        
        '''
            5. 
               5.1 Distance to Kth neighbor
                   Think of this as of quantiles of a distribution
               5.2 Distance to Kth neighbor normalized by 
                   distance to the first neighbor
               
               feat_51, feat_52 are answers to 5.1. and 5.2.
               should be scalars
               
               Do not forget to add self.eps to denominator.
        '''
        for k in self.k_list:
            
            feat_51 = neighs_dist[k-1] # YOUR CODE GOES HERE
            feat_52 = neighs_dist[k-1] / (neighs_dist[0] + self.eps)# YOUR CODE GOES HERE
            
            return_list += [[feat_51, feat_52]]
        
        '''
            6. Mean distance to neighbors of each class for each K from `k_list` 
                   For each class select the neighbors of that class among K nearest neighbors 
                   and compute the average distance to those objects
                   
                   If there are no objects of a certain class among K neighbors, set mean distance to 999
                   
               You can use `np.bincount` with appropriate weights
               Don't forget, that if you divide by something, 
               You need to add `self.eps` to denominator.
        '''
        for k in self.k_list:
            
            # YOUR CODE GOES IN HERE
            bincount = np.bincount(neighs_y[:k], minlength=self.n_classes)
            feats = np.where(
                bincount,
                np.bincount(neighs_y[:k], weights=neighs_dist[:k], minlength=self.n_classes) / (bincount+self.eps),
                999
            )
            
            assert len(feats) == self.n_classes
            return_list += [feats]
        
        
        # merge
        knn_feats = np.hstack(return_list)
        
        assert knn_feats.shape == (239,) or knn_feats.shape == (239, 1)
        return knn_feats

## Sanity check

To make sure you've implemented everything correctly we provide you the correct features for the first 50 objects.

In [ ]:
# a list of K in KNN, starts with one 
k_list = [3, 8, 32]

# Load correct features
true_knn_feats_first50 = np.load('knn_feats_test_first50.npy')

# Create instance of our KNN feature extractor
NNF = NearestNeighborsFeats(n_jobs=1, k_list=k_list, metric='minkowski')

# Fit on train set
NNF.fit(X, Y)

# Get features for test
test_knn_feats = NNF.predict(X_test[:50])

# This should be zero
print ('Deviation from ground thruth features: %f' % np.abs(test_knn_feats - true_knn_feats_first50).sum())

deviation =np.abs(test_knn_feats - true_knn_feats_first50).sum(0)
for m in np.where(deviation > 1e-3)[0]: 
    p = np.where(np.array([87, 88, 117, 146, 152, 239]) > m)[0][0]
    print ('There is a problem in feature %d, which is a part of section %d.' % (m, p + 1))

Deviation from ground thruth features: 0.000000


Now implement parallel computations and compute features for the train and test sets. 

## Get features for test

Now compute features for the whole test set.

In [ ]:
%%time
import os
from tqdm import tqdm_notebook
from multiprocessing import cpu_count

for metric in (['minkowski', 'cosine']):
    print (metric)
    
    # Create instance of our KNN feature extractor
    NNF = NearestNeighborsFeats(n_jobs=cpu_count(), k_list=k_list, metric=metric)
    
    # Fit on train set
    NNF.fit(X, Y)

    # Get features for test
    test_knn_feats = NNF.predict(X_test)
    
    # Dump the features to disk
    np.save('knn_feats_%s_test.npy' % metric , test_knn_feats)
  


minkowski
cosine
CPU times: user 1.62 s, sys: 421 ms, total: 2.04 s
Wall time: 1min 23s


## Get features for train

Compute features for train, using out-of-fold strategy.

In [ ]:
%%time

# Differently from other homework we will not implement OOF predictions ourselves
# but use sklearn's `cross_val_predict`
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold

# We will use two metrics for KNN
for metric in (['minkowski', 'cosine']):
    print (metric)
    
    # Set up splitting scheme, use StratifiedKFold
    # use skf_seed and n_splits defined above with shuffle=True
    skf = StratifiedKFold(n_splits=n_splits,
                          shuffle=True,
                          random_state=skf_seed) # YOUR CODE GOES HERE
    
    # Create instance of our KNN feature extractor
    # n_jobs can be larger than the number of cores
    NNF = NearestNeighborsFeats(n_jobs=cpu_count(), k_list=k_list, metric=metric)
    
    # Get KNN features using OOF use cross_val_predict with right parameters
    preds = cross_val_predict(NNF, X, y=Y, cv=skf) # YOUR CODE GOES HERE
    
    # Save the features
    np.save('knn_feats_%s_train.npy' % metric, preds)

minkowski
cosine
CPU times: user 14.8 s, sys: 4.32 s, total: 19.1 s
Wall time: 9min 49s


# Submit

If you made the above cells work, just run the following cell to produce a number to submit.

In [ ]:
s = 0
for metric in ['minkowski', 'cosine']:
    knn_feats_train = np.load('knn_feats_%s_train.npy' % metric)
    knn_feats_test = np.load('knn_feats_%s_test.npy' % metric)
    
    s += knn_feats_train.mean() + knn_feats_test.mean()
    
answer = np.floor(s)
print (answer)

3838.0


Submit!

In [ ]:
from grader import Grader

grader = Grader()

grader.submit_tag('statistic', answer)

STUDENT_EMAIL = "ebube.madu.243204@unn.edu.ng"
STUDENT_TOKEN = "y7tXLhXmO4a2RIMH"
grader.status()

grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Current answer for task statistic is: 3838.0
You want to submit these numbers:
Task statistic: 3838.0
Submitted to Coursera platform. See results on assignment page!
